In [1]:
# Use CC to detect waveforms similar to the twin events

In [2]:
## Import modules

from obspy import read, UTCDateTime, Stream
from utils_cc import template_match_3c, associate_detections, get_timewin_from_template_name, TemplateEvent, TemplateMatches
from utils_preproc import preprocess_stream
from utils_snuffler import read_normal_markers
from os.path import join
from pandas import read_csv, concat

In [3]:
## Inputs
root = "/Volumes/OmanData/geophones_no_prefilt/data"
tempname = "SiblingA12-17-23-35"
phase = "X"
timelen = 0.04 # Length of time window for CC in seconds

freqmin = 10 # Low corner frequency for bandpass filter
freqmax = 450 # High corner frequency for bandpass filter

mincc = 0.7 # Minimum CC coefficient for detection

numdet_min = 4 # Minimum number of detections to be considered a detected event

delta_max = 0.1 # Maximum difference in delta between detections to be considered the same event

In [4]:
## Read the picks of the master event
timewin_temp = get_timewin_from_template_name(tempname)

pkname = f"{phase.lower()}picks_{tempname}.dat"
inpath = join(root, timewin_temp, pkname)
pickdf = read_normal_markers(inpath)

In [5]:
## Read and process the master event

### Read the hour containing the master event
stream_hour = Stream()
for stname in pickdf["station"]:
    stream_hour += read(join(root, timewin_temp, f"*{stname}*SAC"))

### Preprocess the data
stream_hour = preprocess_stream(stream_hour, freqmin, freqmax)

### Cut the master event
stream_mast = Stream()

for i, row in pickdf.iterrows():
    stname = row["station"]
    starttime = UTCDateTime(row["time"])
    stream_mast += stream_hour.select(station=stname).slice(starttime, starttime+timelen)


In [6]:
## Read the time window list
inpath = join(root, "time_windows.dat")
twindf = read_csv(inpath, delimiter=" ")

In [7]:
## Cross-correlate the master event with all the time windows
print(f"Processing {tempname}...")

detdfs = []
for stname in pickdf["station"]:
    stream_temp = stream_mast.select(station=stname)

    
    for timewin in twindf["time_window"]:
        print(f"Processing {stname}, {timewin}...")

        try:
            stream_targ = read(join(root, timewin, f"*{stname}*SAC"))
        except:
            print(f"Cannot find data for {timewin}!")
            continue
        
        stream_targ = preprocess_stream(stream_targ, freqmin, freqmax)
        detdf = template_match_3c(stream_targ, stream_temp, tempname, mincc=mincc)

        if detdf is not None:
            detdfs.append(detdf)

detdf = concat(detdfs)
detdf.sort_values(by=["time"], ascending=True, inplace=True, ignore_index=True)
numdet = len(detdf)
dur = timelen

if freqmax is None:
    outfile = f"ccdetections_{tempname}_highpass{freqmin:.0f}hz_dur{dur:.2f}s_mincc{mincc:.1f}.csv"
else:
    outfile = f"ccdetections_{tempname}_bandpass{freqmin:.0f}-{freqmax:.0f}hz_dur{dur:.2f}s_mincc{mincc:.1f}.csv"

outpath = join(root, timewin_temp, outfile)
detdf["time_shift"] = detdf["time_shift"].dt.total_seconds()
detdf.to_csv(outpath, sep=" ")
print(f"Results saved to {outpath}.")

Processing SiblingA12-17-23-35...
Processing A04, 2020-01-10-11-00-00...


/opt/homebrew/Caskroom/miniforge/base/envs/obspy/lib/python3.11/site-packages/obspy/signal/cross_correlation.py:291: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norm, out=norm)


No detections found.
Processing A04, 2020-01-10-12-00-00...
No detections found.
Processing A04, 2020-01-10-13-00-00...
No detections found.
Processing A04, 2020-01-10-14-00-00...
No detections found.
Processing A04, 2020-01-10-15-00-00...
No detections found.
Processing A04, 2020-01-10-16-00-00...
No detections found.
Processing A04, 2020-01-10-17-00-00...
No detections found.
Processing A04, 2020-01-10-18-00-00...
No detections found.
Processing A04, 2020-01-10-19-00-00...
No detections found.
Processing A04, 2020-01-10-20-00-00...
No detections found.
Processing A04, 2020-01-10-21-00-00...
No detections found.
Processing A04, 2020-01-10-22-00-00...
No detections found.
Processing A04, 2020-01-10-23-00-00...
No detections found.
Processing A04, 2020-01-11-00-00-00...
No detections found.
Processing A04, 2020-01-11-01-00-00...
No detections found.
Processing A04, 2020-01-11-02-00-00...
No detections found.
Processing A04, 2020-01-11-03-00-00...
No detections found.
Processing A04, 202

/opt/homebrew/Caskroom/miniforge/base/envs/obspy/lib/python3.11/site-packages/obspy/signal/cross_correlation.py:291: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norm, out=norm)


1 detections found.
Processing A04, 2020-01-12-10-00-00...


...

0 detections found.
Processing A01, 2020-01-14-05-00-00...


/opt/homebrew/Caskroom/miniforge/base/envs/obspy/lib/python3.11/site-packages/obspy/signal/cross_correlation.py:291: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norm, out=norm)


No detections found.
Processing A01, 2020-01-14-06-00-00...
No detections found.
Processing A01, 2020-01-14-07-00-00...
No detections found.
Processing A01, 2020-01-14-08-00-00...
No detections found.
Processing A01, 2020-01-14-09-00-00...
No detections found.
Processing A01, 2020-01-14-10-00-00...
No detections found.
Processing A01, 2020-01-14-11-00-00...
No detections found.
Processing A01, 2020-01-14-12-00-00...
No detections found.
Processing A01, 2020-01-14-13-00-00...
No detections found.
Processing A01, 2020-01-14-14-00-00...
No detections found.
Processing A01, 2020-01-14-15-00-00...
No detections found.
Processing A01, 2020-01-14-16-00-00...
No detections found.
Processing A01, 2020-01-14-17-00-00...
No detections found.
Processing A01, 2020-01-14-18-00-00...
No detections found.
Processing A01, 2020-01-14-19-00-00...
No detections found.
Processing A01, 2020-01-14-20-00-00...
No detections found.
Processing A01, 2020-01-14-21-00-00...
No detections found.
Processing A01, 202

/opt/homebrew/Caskroom/miniforge/base/envs/obspy/lib/python3.11/site-packages/obspy/signal/cross_correlation.py:291: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(norm, out=norm)


No detections found.
Processing A01, 2020-01-15-06-00-00...
No detections found.
Processing A01, 2020-01-15-07-00-00...
No detections found.
Processing A01, 2020-01-15-08-00-00...
No detections found.
Processing A01, 2020-01-15-09-00-00...
No detections found.
Processing A01, 2020-01-15-10-00-00...
No detections found.
Processing A01, 2020-01-15-11-00-00...
No detections found.
Processing A01, 2020-01-15-12-00-00...
No detections found.
Processing A01, 2020-01-15-13-00-00...
No detections found.
Processing A01, 2020-01-15-14-00-00...
No detections found.
Processing A01, 2020-01-15-15-00-00...
No detections found.
Processing A01, 2020-01-15-16-00-00...
No detections found.
Processing A01, 2020-01-15-17-00-00...
No detections found.
Processing A01, 2020-01-15-18-00-00...
No detections found.
Processing A01, 2020-01-15-19-00-00...
No detections found.
Processing A01, 2020-01-15-20-00-00...
No detections found.
Processing A01, 2020-01-15-21-00-00...
No detections found.
Processing A01, 202

In [8]:
## Associate the detected events
# import importlib
# import cc_utils
# importlib.reload(cc_utils)

#matches = cc_utils.associate_detections(detdf, numdet_min=numdet_min, delta_max=delta_max)
matches = associate_detections(detdf, numdet_min=numdet_min, delta_max=delta_max)
    

There are in total 90 matches.


In [9]:
# Save the detected events
# import importlib
# import cc_utils
# importlib.reload(cc_utils)

stnames = pickdf["station"].tolist()
starttimes = pickdf["time"].tolist()
template = TemplateEvent(tempname, dur, stnames, starttimes)
tempmatch = TemplateMatches(freqmin, freqmax, mincc, numdet_min, template, matches)
# template = cc_utils.TemplateEvent(tempname, dur, stnames, starttimes)
# tempmatch = cc_utils.TemplateMatches(freqmin, freqmax, mincc, numdet_min, template, matches)

if freqmax is None:
    outfile = f"ccmatches_{tempname}_highpass{freqmin:.0f}hz_dur{dur:.2f}s_mincc{mincc:.1f}_mindet{numdet_min:d}.dat"
else:
    outfile = f"ccmatches_{tempname}_bandpass{freqmin:.0f}-{freqmax:.0f}hz_dur{dur:.2f}s_mincc{mincc:.1f}_mindet{numdet_min:d}.dat"

outpath = join(root, timewin_temp, outfile)

tempmatch.write_to_file(outpath)

Template and match information is saved to /Volumes/OmanData/geophones_no_prefilt/data/2020-01-12-17-00-00/ccmatches_SiblingA12-17-23-35_bandpass10-450hz_dur0.04s_mincc0.7_mindet4.dat
